In [8]:
import os
import numpy as np
import cv2 


调用dirp_set_measurement_params函数，可以对测温参数进行配置，配置完成后需要再次运行dirp_measure或dirp_measure_ex才可已得到修改参数后的测温结果。
测温参数包含以下4个类型：

distance，测温距离：
单位：米
范围：1~25，超过25时填写25即可（超过25米时测温精度会有所下降）
说明：待测目标的距离。红外热像仪接收物体自身辐射的红外线生成热像图，距离越远，红外辐射衰减越多。红外热像仪出厂时默认的标定距离一般是固定的，在这个距离测温最精确，距离过近或过远都会增大测温误差。

humidity，湿度：
单位：%
范围：20~100
说明：目标测量环境下的空气相对湿度。请根据实际测量环境配置，默湿度值配置是否准确将影响测温精度，但影响度较小。

emissivity，发射率：
单位：0.01
范围：0.10~1.00
说明：被测物体表面以辐射的形式释放能量相对强弱的能力。可参考“常见物质发射率表”进行配置，由于被测物体表面可能存在腐蚀、氧化等情况，实际发射率值与参考值可能存在一定差异。发射率配置是否准确将影响测温精度，且影响度较大。

reflection，反射温度：
单位：℃
范围：-40.0~500.0
说明：周围环境物体辐射的能量可能被待测目标表面反射，与待测目标辐射一同被相机接收，造成测温误差。如果周围环境没有特别的高温或低温物体，反射温度配置为环境温度即可。反射温度是否配置准确会影响测温精度，数值与环境温度偏差越大影响度越大。


In [6]:

# class 
##tsdk.exe的储存位置
tsdk = r'./DJI_TSDK_linux/utility/bin/linux/release_x64/dji_irp'
##拍摄的rJPG的储存位置
imgPath='../unTreatedImg/'
##处理结果的储存位置
savepath = "../unTreatedImg/"
imgRLocalName ="DJI_0001_R.JPG"

os.makedirs(savepath,exist_ok=True)

##参数根据实际情况设置
distance=25.0
emissivity=0.95
humidity=45
reflection=51.8 


def use_tsdk(tsdk, imgPath, savepath,imgRLocalName):
    print('start')
    imgRpath = os.path.join(imgPath,imgRLocalName)
    imgCoreName = imgRLocalName[:-4]
    
    ##选择的模式是measure输出的结果是温度信息，不是原始信息；
    param = '-s '+ imgRpath + ' -a measure -o ' + savepath + imgCoreName + '.raw' + ' --distance ' + str(distance) + ' --emissivity '+ str(emissivity) +' --humidity '+ str(humidity) + '--reflection ' + str(reflection)
    r_v = os.system(tsdk+' '+param)
    print (r_v)##输出的为tsdk.exe运行的返回值

# use_tsdk(tsdk, imgPath, savepath, imgRLocalName)


In [7]:
class TSDK_TemProcess():
    """
    使用时初始化一个TSDK类
    """
    def __init__(self,imgRpath,imgCoreName,tsdkpath="./DJI_TSDK_linux/utility/bin/linux/release_x64/dji_irp",savePath="../unTreatedImg/",imgPath="../unTreatedImg/"):
        print("the file TSDK extracting is: "+imgRpath,os.path.isfile(imgRpath))
        self.tsdkGenRaw(tsdkpath,imgRpath,savepath)
        self.thermal_np = self.readRawTem(imgRpath[:-4]+".raw",'uint16')
        # print(self.thermal_np.shape)
                
        distance=25.0
        emissivity=0.95
        humidity=45
        reflection=51.8 
        
    
    def tsdkGenRaw(tsdk, imgRpath, savepath,imgCoreName):
        print('--------tsdk start--------')
        ##选择的模式是measure输出的结果是温度信息，不是原始信息；
        param = '-s '+ imgRpath + ' -a measure -o ' + savepath + imgCoreName + '.raw' + ' --distance ' + str(distance) + ' --emissivity '+ str(emissivity) +' --humidity '+ str(humidity) + '--reflection ' + str(reflection)
        r_v = os.system(tsdk+' '+param)
        print (r_v+'\n---------tsdk end---------')##输出的为tsdk.exe运行的返回值
        
    # 利用numpy的fromfile函数读取raw文件，并指定数据格式
    def readRawTem(path,dtype):
        rows=512#图像的行数
        cols=640#图像的列数
        channels =1# 图像的通道数，灰度图为1
        img=np.fromfile(path, dtype)
        img = img.reshape(rows, cols, channels)/10
        return img

    def ShowRawImgR(path,dtype):
        rows=512#图像的行数
        cols=640#图像的列数
        channels =1# 图像的通道数，灰度图为1
        
        img=np.fromfile(path, dtype)
        img = img.reshape(rows, cols, channels)
        # 利用numpy中array的reshape函数将读取到的数据进行重新排列。
        imgToShow=img.astype("uint8")
        # 展示图像
        cv2.namedWindow('Infared image-640*512-8bit')
        cv2.imshow('Infared image-640*512-8bit',imgToShow)
        # # 如果是uint16的数据请先转成uint8。不然的话，显示会出现问题。
        cv2.waitKey()
        cv2.destroyAllWindows()
        
        
    def get_templist(self,points):
        if not points:
            return 0
        tem = 0
        temList = []
        for p in points:
            tem = self.thermal_np[p[0],p[1]]
            temList.append(tem)
        return temList

    # 转换到温度图下的坐标    
    def get_tempsolo(self,point,svrFlag = 1):
        tem = self.thermal_np[point[0],point[1]]
        return tem

    def get_tempsolo_kernal(self,point):
        point = self.transPt(point)
        tem = self.kernalTemp(point)
        return tem
    
    def kernalTemp(self,point,width=3):
        #获取一块像素面积内的温度 卷积处理值（锐化）
        # kernal=np.array([[1,1,1],[1,2,1],[1,1,1]])
        kernal=np.array([1,1,1,1,2,1,1,1,1])
        cnt = 0
        sum = 0
        for y in range(point[1]-width,point[1]+width+1): #高
            if y<0 or y>=256:
                continue
            for x in range(point[0]-width,point[0]+width+1):#宽
                if x<0 or x>=320:
                    continue
                temp = self.thermal_np[y][x] * kernal[cnt]
                sum+=temp
                cnt+=1
        if cnt==0:
            return 0
        return sum/cnt # 均值

In [25]:
def readRawTem(path,dtype):
    rows=512#图像的行数
    cols=640#图像的列数
    channels =1# 图像的通道数，灰度图为1
    img=np.fromfile(path, dtype)
    img = img.reshape(rows, cols, channels)/10
    return img


imgPath='../../uav/unTreatedImg/'
m1R = cv2.imread(imgPath+"DJI_0001_R.JPG")
# m1Raw = readRaw(imgPath+"DJI_0001_R_1.JPG","uint16")
m1Raw = readRawTem(imgPath+"DJI_0001_R_2.raw","uint16")
# cv2.imshow("m1R",m1R)
# cv2.imshow("m1Raw",m1Raw)
# print(m1Raw)


[[[19.2]
  [18.9]
  [19.1]
  ...
  [18.9]
  [19. ]
  [19.3]]

 [[19.3]
  [18.8]
  [18.8]
  ...
  [19.2]
  [19.2]
  [19.1]]

 [[19.2]
  [18.9]
  [17.9]
  ...
  [19.3]
  [19.3]
  [19.4]]

 ...

 [[25.7]
  [25.4]
  [25.4]
  ...
  [19.6]
  [20. ]
  [20.2]]

 [[25.9]
  [25.3]
  [24.7]
  ...
  [19.7]
  [20. ]
  [19.9]]

 [[25.1]
  [24.7]
  [23.9]
  ...
  [20.2]
  [20.5]
  [20.5]]]


In [8]:
from pyexiv2 import Image
import os

def exifrw(exif_path):
        exif_file = os.path.join(exif_path)
        imge = Image(exif_file)
        exif = imge.read_exif()
        print(exif)
exif_path = r"../unTreatedImg/DJI_0002_R.JPG"
exifrw(exif_path)

{'Exif.Image.ImageDescription': 'default', 'Exif.Image.Make': 'DJI', 'Exif.Image.Model': 'ZH20T', 'Exif.Image.Orientation': '1', 'Exif.Image.XResolution': '72/1', 'Exif.Image.YResolution': '72/1', 'Exif.Image.ResolutionUnit': '2', 'Exif.Image.Software': '04.01.03.00', 'Exif.Image.DateTime': '2022:12:20 15:31:43', 'Exif.Image.YCbCrPositioning': '1', 'Exif.Image.ExifTag': '260', 'Exif.Photo.FNumber': '100/100', 'Exif.Photo.ExifVersion': '48 50 51 48', 'Exif.Photo.DateTimeOriginal': '2022:12:20 15:31:43', 'Exif.Photo.DateTimeDigitized': '2022:12:20 15:31:43', 'Exif.Photo.ComponentsConfiguration': '1 2 3 0', 'Exif.Photo.ApertureValue': '0/100', 'Exif.Photo.MaxApertureValue': '0/100', 'Exif.Photo.Flash': '0', 'Exif.Photo.FocalLength': '13500/1000', 'Exif.Photo.MakerNote': '91 115 101 110 115 111 114 95 105 100 58 49 87 57 68 75 52 77 48 48 49 50 49 86 50 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 93', 'Exif.Photo.FlashpixVersion': '48 49 48 48', 'Exif.Photo.ColorSpace': '1', 'Exif.Photo.PixelXDime

In [6]:
from PIL import Image as Image_PIL
def exifread_PIL(exif_path):
    img = Image_PIL.open(exif_path)
    info = img._getexif()
    print(1)
exifread_PIL(exif_path)

1


In [10]:
dict1 ={'Exif.Image.ImageDescription': 'default', 'Exif.Image.Make': 'DJI', 'Exif.Image.Model': 'ZH20T', 'Exif.Image.Orientation': '1', 'Exif.Image.XResolution': '72/1', 'Exif.Image.YResolution': '72/1', 'Exif.Image.ResolutionUnit': '2', 'Exif.Image.Software': '04.01.03.00', 'Exif.Image.DateTime': '2022:12:20 15:31:43', 'Exif.Image.YCbCrPositioning': '1', 'Exif.Image.ExifTag': '260', 'Exif.Photo.FNumber': '100/100', 'Exif.Photo.ExifVersion': '48 50 51 48', 'Exif.Photo.DateTimeOriginal': '2022:12:20 15:31:43', 'Exif.Photo.DateTimeDigitized': '2022:12:20 15:31:43', 'Exif.Photo.ComponentsConfiguration': '1 2 3 0', 'Exif.Photo.ApertureValue': '0/100', 'Exif.Photo.MaxApertureValue': '0/100', 'Exif.Photo.Flash': '0', 'Exif.Photo.FocalLength': '13500/1000', 'Exif.Photo.MakerNote': '91 115 101 110 115 111 114 95 105 100 58 49 87 57 68 75 52 77 48 48 49 50 49 86 50 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 93', 'Exif.Photo.FlashpixVersion': '48 49 48 48', 'Exif.Photo.ColorSpace': '1', 'Exif.Photo.PixelXDimension': '640', 'Exif.Photo.PixelYDimension': '512', 'Exif.Photo.InteroperabilityTag': '680', 'Exif.Iop.InteroperabilityIndex': 'R98', 'Exif.Iop.InteroperabilityVersion': '48 49 48 48', 'Exif.Photo.FileSource': '3', 'Exif.Photo.SceneType': '1', 'Exif.Photo.DigitalZoomRatio': '100/100', 'Exif.Photo.FocalLengthIn35mmFilm': '58', 'Exif.Photo.Contrast': '0', 'Exif.Photo.Saturation': '0', 'Exif.Photo.Sharpness': '0', 'Exif.Photo.DeviceSettingDescription': '0 0 0 0', 'Exif.Photo.BodySerialNumber': '1W9DK4M00121V2', 'Exif.Photo.LensSpecification': '13500/1000 13500/1000 100/100 100/100', 'Exif.Image.GPSTag': '566', 'Exif.GPSInfo.GPSVersionID': '2 3 0 0', 'Exif.GPSInfo.GPSLatitudeRef': 'N', 'Exif.GPSInfo.GPSLatitude': '30/1 46/1 43438/10000', 'Exif.GPSInfo.GPSLongitudeRef': 'E', 'Exif.GPSInfo.GPSLongitude': '120/1 14/1 291169/10000', 'Exif.GPSInfo.GPSAltitudeRef': '0', 'Exif.GPSInfo.GPSAltitude': '108018/1000', 'Exif.GPSInfo.GPSStatus': 'A', 'Exif.GPSInfo.GPSMapDatum': 'WGS-84', 'Exif.Image.XPComment': '0.9.142\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'Exif.Image.XPKeywords': 'single', 'Exif.Thumbnail.Compression': '7', 'Exif.Thumbnail.XResolution': '72/1', 'Exif.Thumbnail.YResolution': '72/1', 'Exif.Thumbnail.ResolutionUnit': '2', 'Exif.Thumbnail.JPEGInterchangeFormat': '1166', 'Exif.Thumbnail.JPEGInterchangeFormatLength': '10313'} 
li = []
for k,v in dict1.items():
    li.append(k)
print(li)

['Exif.Image.ImageDescription', 'Exif.Image.Make', 'Exif.Image.Model', 'Exif.Image.Orientation', 'Exif.Image.XResolution', 'Exif.Image.YResolution', 'Exif.Image.ResolutionUnit', 'Exif.Image.Software', 'Exif.Image.DateTime', 'Exif.Image.YCbCrPositioning', 'Exif.Image.ExifTag', 'Exif.Photo.FNumber', 'Exif.Photo.ExifVersion', 'Exif.Photo.DateTimeOriginal', 'Exif.Photo.DateTimeDigitized', 'Exif.Photo.ComponentsConfiguration', 'Exif.Photo.ApertureValue', 'Exif.Photo.MaxApertureValue', 'Exif.Photo.Flash', 'Exif.Photo.FocalLength', 'Exif.Photo.MakerNote', 'Exif.Photo.FlashpixVersion', 'Exif.Photo.ColorSpace', 'Exif.Photo.PixelXDimension', 'Exif.Photo.PixelYDimension', 'Exif.Photo.InteroperabilityTag', 'Exif.Iop.InteroperabilityIndex', 'Exif.Iop.InteroperabilityVersion', 'Exif.Photo.FileSource', 'Exif.Photo.SceneType', 'Exif.Photo.DigitalZoomRatio', 'Exif.Photo.FocalLengthIn35mmFilm', 'Exif.Photo.Contrast', 'Exif.Photo.Saturation', 'Exif.Photo.Sharpness', 'Exif.Photo.DeviceSettingDescription',